# **Case QuantumFinance - Disciplina NLP - Classificador de chamados**

**Aten√ß√£o:**
- Leia com aten√ß√£o o descritivo do trabalho e as orienta√ß√µes do template.
- O trabalho deve ser entregue respeitando a estrutura do arquivo de template em notebook "Template_Trabalho_Final_NLP.ipynb" e compactado no formato .zip. Apenas um arquivo no formato .ipynb deve ser entregue consolidando todo o trabalho.

***Participantes (RM - NOME):***<br><br>
359735 - J√©ssica Portela de Castro<br>
358404 - Tiago Freire Barbosa<br><br>

###**Crie um classificador de chamados aplicando t√©cnicas de PLN**
---

A **QuantumFinance** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou d√∫vida e depois √© direcionado para alguma √°rea especialista no assunto para uma melhor tratativa.‚Äã

1. Crie ao menos um modelo classificador de assuntos aplicando t√©cnicas de NLP (PLN), Vetoriza√ß√£o (n-grama + m√©trica) e modelo supervisionado, que consiga classificar atrav√©s de um texto o assunto conforme dispon√≠vel na base de dados [1] para treinamento e valida√ß√£o do seu modelo.‚Äã

  O modelo precisar atingir um score na **m√©trica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).‚Äã

2. Utilizar ao menos uma aplica√ß√£o de modelos com Embeddings usando Word2Vec e/ou LLM¬¥s para criar o modelo classificador com os crit√©rios do item 1. N√£o √© necess√°rio implementar aplica√ß√µes usando servi√ßos de API da OpenAI ou outros por exemplo.

Fique √† vontade para testar e explorar as t√©cnicas de pr√©-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decis√µes durante o desenvolvimento.‚Äã

**Composi√ß√£o da nota:‚Äã**

**50%** - Demonstra√ß√µes das aplica√ß√µes das t√©cnicas de PLN (regras, pr√©-processamentos, tratamentos, variedade de modelos aplicados, aplica√ß√µes de GenIA, organiza√ß√£o do pipeline, etc.)‚Äã

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campe√£o (validar com  a M√©trica F1 Score). **Separar o pipeline completo do modelo campe√£o conforme template.‚Äã**

O trabalho poder√° ser feito em grupo de 2 at√© 4 pessoas (mesmo grupo do Startup One) e trabalhos iguais ser√£o descontado nota e pass√≠vel de reprova√ß√£o.

**[1] = ‚Äãhttps://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

**[F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)** com average='weighted'

#### Importa√ß√£o das Bibliotecas

Necess√°rio executar, reiniciar o ambiente e executar novamente para correto funcionamento do gensim<br>

Centralizamos nesta c√©lula todas as importa√ß√µes, para melhorar a organiza√ß√£o do c√≥digo<br>

Posteriormente no modelo campe√£o ser√£o importadas apenas as bibliotecas necess√°ria, para execu√ß√£o de forma independente

In [1]:
!pip install gensim==4.3.2 scipy==1.10.1 numpy==1.26.4 --quiet
!python -m spacy download pt_core_news_sm --quiet

import pandas as pd
import nltk
import string
import warnings
import spacy
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from nltk.tokenize import word_tokenize

import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

warnings.filterwarnings('ignore')

     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 13.0/13.0 MB 75.5 MB/s eta 0:00:00
‚úî Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
‚ö† Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#### Importa√ß√£o de Dataset

In [2]:
df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

#### An√°lise Explorat√≥ria de Dados

- O dataset possui **21072** linhas e **4** colunas

- N√£o possui valores nulos

- Os tipos de dados est√£o distrubu√≠dos entre **int64** e **Object**

In [3]:
df.info()
print('\n')
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB




(21072, 4)

Visualiza√ß√£o das linhas iniciais

In [4]:
df.head(10)

id_reclamacao              data_abertura  \
0        3229299  2019-05-01T12:00:00-05:00   
1        3199379  2019-04-02T12:00:00-05:00   
2        3233499  2019-05-06T12:00:00-05:00   
3        3180294  2019-03-14T12:00:00-05:00   
4        3224980  2019-04-27T12:00:00-05:00   
5        3209411  2019-04-11T12:00:00-05:00   
6        3331023  2019-08-06T12:00:00-05:00   
7        3352857  2019-08-24T12:00:00-05:00   
8        3226110  2019-04-29T12:00:00-05:00   
9        3237765  2019-05-09T12:00:00-05:00   

                             categoria  \
0              Hipotecas / Empr√©stimos   
1  Cart√£o de cr√©dito / Cart√£o pr√©-pago   
2  Cart√£o de cr√©dito / Cart√£o pr√©-pago   
3  Cart√£o de cr√©dito / Cart√£o pr√©-pago   
4           Servi√ßos de conta banc√°ria   
5  Cart√£o de cr√©dito / Cart√£o pr√©-pago   
6           Servi√ßos de conta banc√°ria   
7                               Outros   
8         Roubo / Relat√≥rio de disputa   
9  Cart√£o de cr√©dito / Cart√£o pr√©-pago   

                                descricao_reclamacao  
0  Bom dia, meu nome √© xxxx xxxx e agrade√ßo se vo...  
1  Atualizei meu cart√£o xxxx xxxx em xx/xx/2018 e...  
2  O cart√£o Chase foi relatado em xx/xx/2019. No ...  
3  Em xx/xx/2018, enquanto tentava reservar um ti...  
4  Meu neto me d√™ cheque por {$ 1600,00} Eu depos...  
5                       Voc√™ pode remover a consulta  
6  Sem aviso pr√©vio J.P. Morgan Chase restringiu ...  
7  Durante os meses de ver√£o, experimento uma ren...  
8  Em xxxx xx/xx/2019, fiz um pagamento {$ 300.00...  
9  Eu tenho um cart√£o de cr√©dito Chase que est√° r...

Visualizando texto do campo de descri√ß√£o da reclama√ß√£o, aqui notamos que o texto possui pontua√ß√£o, caracteres em diferentes tamanhos, mascaramento de dados, e stopwords evidenciando a necessidade de normaliza√ß√£o e pr√©-processamento destes textos

In [5]:
df['descricao_reclamacao'][6]

'Sem aviso pr√©vio J.P. Morgan Chase restringiu minha conta pelo meu cart√£o de d√©bito ter√ßa -feira xx/xx/2019. Na quinta -feira, xx/xx/2019, entrei em uma filial depois de ser aconselhado por um representante de atendimento ao cliente de que minha conta seria realmente fechada. Entrei na filial para ver como posso remover os fundos que est√£o l√° atualmente, como se meu dep√≥sito direto do meu local de trabalho fosse devolvido. O Associado do Banco e o Representante de Atendimento ao Cliente me garantiram que os fundos publicariam, mas eles podem levar um dia √∫til adicional. Naquele s√°bado, tentei entrar em uma filial mais uma vez para recuperar meus fundos que realmente publicaram na sexta -feira, xx/xx/2019. Ao olhar para minha conta, percebo que os fundos foram revertidos e n√£o estavam mais presentes na minha declara√ß√£o atual. Fui chamado e me disseram que pode levar dois dias √∫teis informando na ter√ßa -feira, xx/xx/xxxx seria a data em que meus fundos estariam dispon√≠vei

In [6]:
df['descricao_reclamacao'][8]

'Em xxxx xx/xx/2019, fiz um pagamento {$ 300.00} a um varejista on -line usando o Chase Quick Pay com xxxx no site da Chase. Percebi que este era um site de fraudes ap√≥s nenhuma confirma√ß√£o, produto ou resposta a quaisquer perguntas sobre o pedido e contatou o Chase para obter mais informa√ß√µes sobre a transa√ß√£o para tentar obter um reembolso pelo banco de varejistas.\r\n\r\nEntrei em contato com o Chase por meio de uma mensagem segura em xx/xx/2019 explicando o que aconteceu e perguntei: "Existe uma maneira de reverter essa transa√ß√£o, ou voc√™ tem um contato no XXXX que pode me fornecer mais informa√ß√µes sobre o destinat√°rio? \'\' Que xxxx , minha mensagem foi revisada e recebi uma resposta reafirmando meu e -mail original e me informando que "a transfer√™ncia foi conclu√≠da. No entanto, como voc√™ mencionou que o site √© uma farsa enquanto podemos lidar com a maioria das perguntas por e -mail, algumas exigem a experi√™ncia de outra equipe. Para ajud√° -lo melhor em rela√ß√£

No dataset que existe uma distribui√ß√£o de categorias similar entre **Servi√ßos de conta banc√°ria**, **Cart√£o de cr√©dito / Cart√£o pr√©-pago** e **Roubo / Relat√≥rio de disputa**, enquanto **Hipotecas / Empr√©stimos** e **Outros** encontram-se um pouco distante das demais categorias em quest√£o de quantidade de amostras

In [7]:
df['categoria'].value_counts(ascending=True).plot(kind='barh')

<Axes: ylabel='categoria'>

Bom desenvolvimento!

###**Area de desenvolvimento e valida√ß√µes**

Fa√ßa aqui as demonstra√ß√µes das aplica√ß√µes das t√©cnicas de PLN (regras, pr√©-processamentos, tratamentos, variedade de modelos aplicados, organiza√ß√£o do pipeline, etc.)‚Äã

Fique √† vontade para testar e explorar as t√©cnicas de pr√©-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decis√µes durante o desenvolvimento.‚Äã

#### DecisionTreeClassifier

Nos pipelines abaixo, testaremos inicialmente com o modelo **DecisionTreeClassifier**, posteriomente implementaremos demais modelos de classifica√ß√£o com o pipeline de melhor resultado para compara√ß√£o

##### Pipeline 01 - Vetoriza√ß√£o BOW + Unigrama + Remo√ß√£o de Stopwords

- Este pipeline realiza a classifica√ß√£o de textos com base em vetoriza√ß√£o Bag of Words (BoW), utilizando unigramas e remo√ß√£o de stopwords

- os dados s√£o copiados e divididos em conjuntos de treino e teste, com 75% dos dados para treino e 25% para teste

- Em seguida, s√£o baixadas as listas de stopwords do NLTK em portugu√™s, sendo que duas palavras adicionais ("xx" e "xxxx") s√£o inclu√≠das manualmente para remo√ß√£o posterior

- Para o pr√©-processamento, s√£o definidas duas fun√ß√µes: uma para remo√ß√£o de pontua√ß√£o e outra que realiza a normaliza√ß√£o completa do texto. Essa normaliza√ß√£o inclui convers√£o para min√∫sculas, remo√ß√£o de pontua√ß√£o e d√≠gitos, tokeniza√ß√£o com word_tokenize, remo√ß√£o de stopwords e de palavras com menos de tr√™s caracteres. O texto limpo √© ent√£o reconstru√≠do como uma string normalizada

- Ap√≥s a limpeza, o vetor de texto √© criado com a ferramenta CountVectorizer, configurada para gerar unigramas (ou seja, palavras isoladas) e eliminar as stopwords customizadas. O CountVectorizer √© ajustado somente com os dados de treino, garantindo que o vocabul√°rio do modelo n√£o incorpore informa√ß√µes do conjunto de teste

- Com o vocabul√°rio aprendido, os textos de treino e teste s√£o transformados em vetores num√©ricos esparsos, representando a frequ√™ncia das palavras em cada documento. Esses vetores s√£o usados para treinar um modelo do tipo DecisionTreeClassifier, que tenta aprender padr√µes entre os vetores de texto e suas respectivas categorias

- Ao final, o modelo realiza previs√µes no conjunto de teste, e a performance √© avaliada utilizando a m√©trica F1 Score com m√©dia ponderada (average='weighted'), adequada para situa√ß√µes em que as classes est√£o desbalanceadas

In [8]:
df_pipeline_01 = df.copy()

df_train, df_test = train_test_split(
    df_pipeline_01,
    test_size=0.25,
    random_state=42
)

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

stopwords = nltk.corpus.stopwords.words('portuguese')

stopwords_new = stopwords + ["xx", "xxxx"]

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords_new]
    text = [y for y in text if len(y) > 2]
    return " ".join([t for t in text])

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(normalize_str)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(normalize_str)

vect = CountVectorizer(ngram_range=(1,1), stop_words=stopwords_new)
vect.fit(df_train.descricao_reclamacao)

x_train = vect.transform(df_train.descricao_reclamacao)
y_train = df_train["categoria"]

model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

x_test = vect.transform(df_test.descricao_reclamacao)

y_prediction = model.predict(x_test)

y_test = df_test["categoria"]
f1 = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1)

F1 Score: 0.7309240224346284


##### Pipeline 02 - Vetoriza√ß√£o BOW + Unigrama & Bigrama + Remo√ß√£o de Stopwords

Pipeline semelhante ao anterior com a diferen√ßa que neste, implementamos unigramas e bigramas no **CountVectorizer**

In [9]:
df_pipeline_02 = df.copy()

df_train, df_test = train_test_split(
    df_pipeline_02,
    test_size=0.25,
    random_state=42
)

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

stopwords = nltk.corpus.stopwords.words('portuguese')

stopwords_new = stopwords + ["xx", "xxxx"]

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords_new]
    text = [y for y in text if len(y) > 2]
    return " ".join([t for t in text])

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(normalize_str)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(normalize_str)

vect = CountVectorizer(ngram_range=(1,2), stop_words=stopwords_new)
vect.fit(df_train.descricao_reclamacao)

x_train = vect.transform(df_train.descricao_reclamacao)
y_train = df_train["categoria"]

model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

x_test = vect.transform(df_test.descricao_reclamacao)

y_prediction = model.predict(x_test)

y_test = df_test["categoria"]
f1 = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1)

F1 Score: 0.7454976711454935


##### Pipeline 03 - Vetoriza√ß√£o TF-IDF + Lematiza√ß√£o + Unigrama + Remo√ß√£o de Stopwords

- Pipeline semelhante aos anteriores com a diferen√ßa de que neste implementamos a fun√ß√£o **lemmatizer_text** para efetuar a lematiza√ß√£o com a biblioteca Spacy<br>

- Aqui utilizamos TF-IDF para vetoriza√ß√£o, com unigrama e remo√ß√£o de stopwords

In [10]:
df_pipeline_03 = df.copy()

df_train, df_test = train_test_split(
    df_pipeline_03,
    test_size=0.25,
    random_state=42
)

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nlp = spacy.load('pt_core_news_sm')

stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords_new = stopwords + ["xx", "xxxx"]

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords_new]
    text = [y for y in text if len(y) > 2]
    return " ".join([t for t in text])

def lemmatizer_text(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(normalize_str)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(normalize_str)

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(lemmatizer_text)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(lemmatizer_text)

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True, stop_words=stopwords_new)
vect.fit(df_train.descricao_reclamacao)

x_train = vect.transform(df_train.descricao_reclamacao)
y_train = df_train["categoria"]

model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

x_test = vect.transform(df_test.descricao_reclamacao)

y_prediction = model.predict(x_test)

y_test = df_test["categoria"]
f1 = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1)

F1 Score: 0.757195929538738


##### Pipeline 04 - Vetoriza√ß√£o TF-IDF + Lematiza√ß√£o + Unigrama & Bigrama + Remo√ß√£o de Stopwords

Neste pipeline implementamos l√≥gica semelhante aos demais, por√©m neste efetuamos a remo√ß√£o de stopwords do NLTK e Spacy<br>

Aqui implementamos vetoriza√ß√£o com **TF-IDF**, utilizando unigramas e bigramas<br>

De todos os modelos at√© o momento este atingiu o maior **F1-Score**

In [11]:
df_pipeline_04 = df.copy()

df_train, df_test = train_test_split(
    df_pipeline_04,
    test_size=0.25,
    random_state=42
)

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nlp = spacy.load('pt_core_news_sm')

stopwords = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))
stopwords_new = stopwords + ["xx", "xxxx"]

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords_new]
    text = [y for y in text if len(y) > 2]
    return " ".join([t for t in text])

def lemmatizer_text(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(normalize_str)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(normalize_str)

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(lemmatizer_text)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(lemmatizer_text)

vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=stopwords_new)
vect.fit(df_train.descricao_reclamacao)

x_train = vect.transform(df_train.descricao_reclamacao)
y_train = df_train["categoria"]

model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

x_test = vect.transform(df_test.descricao_reclamacao)

y_prediction = model.predict(x_test)

y_test = df_test["categoria"]
f1_tree = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1_tree)

F1 Score: 0.7627874466001402


###### LogisticRegression

In [12]:
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

x_test = vect.transform(df_test.descricao_reclamacao)

y_prediction = model.predict(x_test)

y_test = df_test["categoria"]
f1_lr = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1_lr)

F1 Score: 0.9073597218887184


###### NaiveBayes

In [13]:
model = MultinomialNB()
model.fit(x_train, y_train)

x_test = vect.transform(df_test.descricao_reclamacao)

y_prediction = model.predict(x_test)

y_test = df_test["categoria"]
f1_nb = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1_nb)

F1 Score: 0.7319836368831959


###### RandomForest

In [14]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

x_test = vect.transform(df_test.descricao_reclamacao)

y_prediction = model.predict(x_test)

y_test = df_test["categoria"]
f1_rf = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1_rf)

F1 Score: 0.8063001769268272


#### Resultados

A partir do **pipeline 04**, modelo que atingiu maior F1-Score com o modelo **DecisionClassifier**, treinamos 3 outros modelos de classifica√ß√£o, sendo que entre todos **LogisticRegression** atingiu o maior F1-Score

In [15]:
nomes = ['DecisionTree', 'LogisticRegression', 'NaiveBayes', 'RandomForest']

f1_scores = [f1_tree, f1_lr, f1_nb, f1_rf]

plt.figure(figsize=(8,5))
plt.bar(nomes, f1_scores)
plt.title('Compara√ß√£o de F1-Score entre Pipelines')
plt.ylabel('F1-Score')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)


for i, v in enumerate(f1_scores):
    plt.text(i, v + 0.02, f'{v:.2f}', ha='center')

plt.show()

#### DecisionTreeClassifier + Embeddings Word2Vec

Download e descompacta√ß√£o do arquivo de embeddings

In [16]:
!wget 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/skip_s300.zip'

!unzip 'skip_s300.zip'

--2025-07-14 23:34:35--  https://dados-ml-pln.s3-sa-east-1.amazonaws.com/skip_s300.zip
Resolving dados-ml-pln.s3-sa-east-1.amazonaws.com (dados-ml-pln.s3-sa-east-1.amazonaws.com)... 3.5.232.253, 52.95.165.27, 3.5.234.43, ...
Connecting to dados-ml-pln.s3-sa-east-1.amazonaws.com (dados-ml-pln.s3-sa-east-1.amazonaws.com)|3.5.232.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 958619745 (914M) [application/zip]
Saving to: ‚Äòskip_s300.zip‚Äô

skip_s300.zip       100%[===================>] 914.21M  20.6MB/s    in 49s     

2025-07-14 23:35:25 (18.7 MB/s) - ‚Äòskip_s300.zip‚Äô saved [958619745/958619745]

Archive:  skip_s300.zip
  inflating: skip_s300.txt           


In [17]:
model_skip = KeyedVectors.load_word2vec_format('skip_s300.txt')

##### Pipeline 05 - Vetoriza√ß√£o TF-IDF + Lematiza√ß√£o + Unigrama & Bigrama + Remo√ß√£o de Stopwords

- Este pipeline utiliza vetores de palavras com Word2Vec (modelo model_skip) e uma √°rvore de decis√£o como classificador

- Primeiramente, ele realiza uma c√≥pia do DataFrame original e divide os dados em conjuntos de treino e teste, com 25% dos dados reservados para teste e 75% para treino

- Em seguida, s√£o baixados recursos do NLTK e carregado o modelo da l√≠ngua portuguesa do spaCy. As stopwords padr√£o do spaCy e do NLTK s√£o combinadas em uma √∫nica lista, acrescida de termos adicionais ("xx", "xxxx")

- O texto √© ent√£o normalizado por meio de fun√ß√µes que convertem tudo para min√∫sculas, removem pontua√ß√µes, d√≠gitos, stopwords e palavras curtas.

- Ap√≥s a normaliza√ß√£o, cada descri√ß√£o de reclama√ß√£o √© transformada em um vetor num√©rico atrav√©s da fun√ß√£o **average_vector**, calculando-se a m√©dia dos vetores Word2Vec das palavras presentes no texto

- Com os dados vetorizados, o modelo DecisionTreeClassifier √© treinado com os vetores do conjunto de treino e suas respectivas categorias

- Posteriormente, o modelo √© testado com os dados de teste, e o desempenho √© avaliado utilizando a m√©trica F1-Score ponderada

- Escolhemos o modelo skip_s300 para classifica√ß√£o porque ele gera vetores de palavras mais ricos semanticamente, capturando melhor o contexto mesmo em textos curtos ou com palavras menos frequentes, o que melhora o desempenho do classificador em tarefas de categoriza√ß√£o textual

In [18]:
df_pipeline_05 = df.copy()

df_train, df_test = train_test_split(
    df_pipeline_05,
    test_size=0.25,
    random_state=42
)

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nlp = spacy.load('pt_core_news_sm')

stopwords = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))
stopwords_new = stopwords + ["xx", "xxxx"]

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords_new]
    text = [y for y in text if len(y) > 2]
    return " ".join([t for t in text])

vectorizer = model_skip

def average_vector(words):
  vectors = [vectorizer[word] for word in words if word in vectorizer]
  if vectors:
    return sum(vectors) / len(vectors)
  else:
    vector_size = vectorizer.vector_size
    wv_res = np.zeros(vector_size)
    return wv_res

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(normalize_str)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(normalize_str)

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(average_vector)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(average_vector)

x_train = df_train['descricao_reclamacao']
y_train = df_train["categoria"]

model = DecisionTreeClassifier(random_state=42)
model.fit(list(x_train), y_train)

x_test = df_test['descricao_reclamacao']
y_test = df_test["categoria"]

y_prediction = model.predict(list(x_test))

f1_tree_w2v = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1_tree_w2v)

F1 Score: 0.4368768851511542


###### LogisticRegression

In [19]:
model = LogisticRegression(random_state=42)
model.fit(list(x_train), y_train)

x_test = df_test['descricao_reclamacao']
y_test = df_test["categoria"]

y_prediction = model.predict(list(x_test))

f1_lr_w2v = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1_lr_w2v)

F1 Score: 0.5757857611108066


###### RandomForest

In [20]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(list(x_train), y_train)

x_test = df_test['descricao_reclamacao']
y_test = df_test["categoria"]

y_prediction = model.predict(list(x_test))

f1_rf_w2v = f1_score(y_test, y_prediction,average='weighted')
print("F1 Score:", f1_rf_w2v)

F1 Score: 0.5871349438125817


#### Resultados

Os modelos com Word2Vec apresentaram desempenho inferior em compara√ß√£o com modelos anteriores

In [22]:
nomes = ['DecisionTree', 'LogisticRegression', 'RandomForest']

f1_scores = [f1_tree_w2v, f1_lr_w2v, f1_rf_w2v]

plt.figure(figsize=(8,5))
plt.bar(nomes, f1_scores)
plt.title('Compara√ß√£o de F1-Score entre Pipelines')
plt.ylabel('F1-Score')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)


for i, v in enumerate(f1_scores):
    plt.text(i, v + 0.02, f'{v:.2f}', ha='center')

plt.show()

###**Valida√ß√£o do professor**

Consolidar apenas os scripts do seu **modelo campe√£o**, desde o carregamento do dataframe, separa√ß√£o das amostras, tratamentos utilizados (fun√ß√µes, limpezas, etc.), cria√ß√£o dos objetos de vetoriza√ß√£o dos textos e modelo treinado e outras implementa√ß√µes utilizadas no processo de desenvolvimento do modelo.

O modelo precisar atingir um score na m√©trica F1 Score superior a 75%.

**Aten√ß√£o:**
- **Implemente aqui apenas os scripts que fazem parte do modelo campe√£o.**
- **Execute o pipeline do modelo campe√£o completamente para garantir que n√£o tet√° erros no script.**


#### Modelo Campe√£o üèÜ - Vetoriza√ß√£o TF-IDF + Lematiza√ß√£o + Unigrama & Bigrama + Remo√ß√£o de Stopwords + LogisticRegression

- Nosso pipeline campe√£o implementa classifica√ß√£o de texto, utilizando pr√©-processamento lingu√≠stico com spaCy e NLTK, vetoriza√ß√£o com TF-IDF bigramas e classifica√ß√£o com Regress√£o Log√≠stica

- Inicialmente, o modelo de linguagem pt_core_news_sm do spaCy √© baixado para permitir a lematiza√ß√£o de textos em portugu√™s. Em seguida, s√£o importadas bibliotecas relevantes e suprimidos avisos para manter o notebook mais limpo

- O conjunto de dados √© copiado e dividido em treino e teste (75%/25%). Recursos como stopwords s√£o carregados tanto do spaCy quanto do NLTK e combinados em uma lista estendida com termos extras ("xx", "xxxx").

- Fun√ß√µes de limpeza textual s√£o aplicadas para: remover pontua√ß√£o e n√∫meros, tokenizar palavras, eliminar stopwords e palavras curtas, lematizar os tokens com o modelo do spaCy, transformando palavras para suas formas base

- Ap√≥s o pr√©-processamento, os textos s√£o vetorizados com TfidfVectorizer configurado para unigramas e bigramas, e com remo√ß√£o de stopwords personalizada

- O modelo Logistic Regression √© treinado com os dados vetorizados de treino

- Por fim, o modelo √© testado com os dados de teste transformados, e a performance √© avaliada por meio do F1-Score ponderado, que √© impresso como m√©trica final da classifica√ß√£o

In [23]:
!python -m spacy download pt_core_news_sm --quiet

import pandas as pd
import nltk
import string
import warnings
import spacy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from nltk.tokenize import word_tokenize

warnings.filterwarnings('ignore')

df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')

df_pipeline_campeao = df.copy()

df_train, df_test = train_test_split(
    df_pipeline_campeao,
    test_size=0.25,
    random_state=42
)

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nlp = spacy.load('pt_core_news_sm')

stopwords = list(set(nlp.Defaults.stop_words).union(set(nltk.corpus.stopwords.words('portuguese'))))
stopwords_new = stopwords + ["xx", "xxxx"]

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = "".join([w for w in text if not w.isdigit()])
    text = word_tokenize(text)
    text = [x for x in text if x not in stopwords_new]
    text = [y for y in text if len(y) > 2]
    return " ".join([t for t in text])

def lemmatizer_text(text):
  sent = []
  doc = nlp(text)
  for word in doc:
      sent.append(word.lemma_)
  return " ".join(sent)

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(normalize_str)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(normalize_str)

df_train['descricao_reclamacao'] = df_train['descricao_reclamacao'].apply(lemmatizer_text)

df_test['descricao_reclamacao'] = df_test['descricao_reclamacao'].apply(lemmatizer_text)

vect = TfidfVectorizer(ngram_range=(1,2), use_idf=True, stop_words=stopwords_new)
vect.fit(df_train.descricao_reclamacao)

x_train = vect.transform(df_train.descricao_reclamacao)
y_train = df_train["categoria"]

model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

x_test = vect.transform(df_test.descricao_reclamacao)

y_prediction = model.predict(x_test)

y_test = df_test["categoria"]
f1_lr_campeao = f1_score(y_test, y_prediction,average='weighted')
print("\n\nF1 Score:", f1_lr_campeao)

     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 13.0/13.0 MB 41.0 MB/s eta 0:00:00
‚úî Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
‚ö† Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


F1 Score: 0.9073597218887184
